In [1]:
# !pip install antropy

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.signal import butter,filtfilt
import scipy.signal
from scipy.stats import skew,kurtosis
import antropy as an
from numpy import mean
from numpy import std
import time

In [2]:
channels = ["AF3", "F7", "F3", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F4", "F8", "AF4"]

In [3]:
# Filter requirements
T = 150 # sample period
fs = 128 # sample rate (Heartz)
cutoff = 40 # desired cutoff frequency of filter
nyq = 0.5* fs
order = 1 # sin wave can be approx represented as quadratic
n = int(T * fs) # total number of samples

t = np.linspace(0, T, n, endpoint=False)

normal_cutoff = [3/nyq, cutoff/nyq]
b,a = butter(order, normal_cutoff, btype='band', analog=False)
SS = 0
TIMES = np.zeros([48])

In [4]:
Features = [
    "mean_PSD","STD_PSD","A_mean",
    "A_STD","A_Var","A_range",
    "A_skew","A_kurtosis","Permutation_E",
    "Spectral_E","SVD_E","Approximate_E",
    "Sample_E","Petrosian_FD","Katz_FD",
    "Higuchi_FD",
    "Detrended fluctuation analysis",
    "Patient Number",
    "Label"
] 

## Feature Extraction

In [5]:
file = f"data/sub01_hi.txt"
df = pd.read_csv(file,header=None,sep="   ")
df

C:\Users\nimis\AppData\Local\Temp\ipykernel_19960\2489124467.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file,header=None,sep="   ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,4584.62,3902.05,4571.79,4589.23,4124.62,3825.13,4152.82,4579.49,4690.77,4260.00,4027.18,4385.13,4480.51,4230.77
1,4584.10,3895.90,4574.87,4567.69,4124.10,3827.18,4157.95,4585.13,4695.38,4268.21,4034.36,4380.00,4501.54,4197.44
2,4574.36,3893.85,4576.92,4572.82,4123.59,3829.23,4165.13,4590.26,4702.56,4281.54,4030.77,4366.67,4521.03,4176.41
3,4573.85,3906.15,4572.82,4612.31,4137.95,3830.77,4167.18,4596.92,4706.15,4285.64,4038.46,4376.41,4518.97,4207.18
4,4583.59,3911.28,4570.26,4621.03,4150.77,3833.85,4166.15,4597.44,4705.13,4282.05,4051.79,4387.18,4520.51,4220.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19195,4593.33,3933.85,4570.77,4391.28,4217.95,3845.64,4148.72,4603.59,4748.72,4311.28,4144.10,4488.72,4498.46,4328.21
19196,4602.05,3960.00,4577.44,4405.13,4217.44,3844.10,4148.72,4599.49,4743.08,4305.13,4135.38,4490.77,4495.38,4327.18
19197,4605.64,3987.69,4573.85,4414.36,4218.46,3833.85,4136.92,4585.64,4727.69,4292.82,4118.97,4488.21,4484.62,4322.05
19198,4606.15,3992.82,4572.82,4417.44,4217.44,3830.26,4132.31,4581.54,4718.46,4283.59,4111.79,4495.38,4470.26,4314.87


### Reading Data and extracting features

In [6]:
# start_time = time.time()
features_df = pd.DataFrame(columns=Features)
ss = 0
conditions = {"hi":1,"lo":0}
for condition in conditions.keys():
    for s in range(1,49):
        file = f"data/sub{s:02}_{condition}.txt"
        df = pd.read_csv(file,header=None,sep="   ",engine='python')
        df.columns = channels
        for channel in channels:
            df.insert(0," ".join([channel,"Filtered"]),filtfilt(b,a,df[channel].values))
            df = df.drop(channel,axis=1)
            df.insert(0," ".join([channel,"Despiked"]),df[" ".join([channel,"Filtered"])].where(df[" ".join([channel,"Filtered"])]<df[" ".join([channel,"Filtered"])].quantile(0.97),df[" ".join([channel,"Filtered"])].mean()))
            df = df.drop(" ".join([channel,"Filtered"]),axis=1)
            df.insert(0,channel,df[" ".join([channel,"Despiked"])].where(df[" ".join([channel,"Despiked"])].quantile(0.05)<df[" ".join([channel,"Despiked"])],df[" ".join([channel,"Despiked"])].mean()))
            df = df.drop(" ".join([channel,"Despiked"]),axis=1)
        w = 0
        for window in range(0,60):
            eeg_window = df[w:w+640] # windowing, win len : 5 sec, overalp :2.5 sec
            w = w+320
            features = np.zeros([300])
            features_channel = np.zeros([19])
            i=0
            for i in range(14):
                channel = eeg_window[channels[i]]

                #PSD features
                f,pxx = scipy.signal.welch(channel,fs) # extract psd according to welch method
                features[i] = mean(pxx)
                features[i+14] = std(pxx)
                
                #Statistical features
                features[i+28] = mean(channel)
                features[i+42] = std(channel)
                features[i+56] = np.var(channel)
                features[i+70] = max(channel)-min(channel)
                features[i+84] = skew(channel)
                features[i+98] = kurtosis(channel)

                #Entropy features
                features[i+112]= an.perm_entropy(channel, order=3, normalize=True)                 
                features[i+126]= an.spectral_entropy(channel, 100, method='welch', normalize=True) 
                features[i+140]= an.svd_entropy(channel, order=3, delay=1, normalize=True)         
                features[i+154]= an.app_entropy(channel, order=2, metric='chebyshev')              
                features[i+168]= an.sample_entropy(channel, order=2, metric='chebyshev')

                #Fractal Dimensions Features
                features[i+182] = an.petrosian_fd(channel)
                features[i+196] = an.katz_fd(channel)
                features[i+210] = an.higuchi_fd(channel)
                features[i+224] = an.detrended_fluctuation(channel)
            features_channel[0] = mean(features[0:14]) # Mean of PSD
            features_channel[1] = mean(features[14:28]) # Standered Deviation of PSD
            features_channel[2] = mean(features[28:42]) # Amplitude Mean
            features_channel[3] = mean(features[42:56]) # Amplitude Standered Deviation
            features_channel[4] = mean(features[56:70]) # Amplitude variance
            features_channel[5] = mean(features[70:84]) # Amplitude Range
            features_channel[6] = mean(features[84:98]) # Amplitude Skew
            features_channel[7] = mean(features[98:112]) # Amplitude kurtosis
            features_channel[8] = mean(features[112:126]) # Permutation entropy
            features_channel[9] = mean(features[126:140]) # Spectral entropy
            features_channel[10] = mean(features[140:154]) # Singular value decomposition entropy
            features_channel[11] = mean(features[154:168]) # Approximate entropy
            features_channel[12] = mean(features[168:182]) # Sample entropy
            features_channel[13] = mean(features[182:196]) # Petrosian fractal dimension
            features_channel[14] = mean(features[196:210]) # Katz fractal dimension
            features_channel[15] = mean(features[210:224]) # Higuchi fractal dimension
            features_channel[16] = mean(features[224:238]) # Detrended fluctuation analysis
            features_channel[17] = s-1
            features_channel[18] = conditions[condition]
            # features_df = features_df._append(pd.DataFrame([features_channel],columns=Features),ignore_index=True)
            features_df.loc[ss] = features_channel
            ss = ss+1

In [7]:
features_df

,mean_PSD,STD_PSD,A_mean,A_STD,A_Var,A_range,A_skew,A_kurtosis,Permutation_E,Spectral_E,SVD_E,Approximate_E,Sample_E,Petrosian_FD,Katz_FD,Higuchi_FD,Detrended fluctuation analysis,Patient Number,Label
0,0.666708,1.129708,0.522807,6.610550,46.226834,35.296581,0.269685,0.063777,0.921589,0.826680,0.879714,1.297318,1.426704,1.024025,3.909459,1.712382,0.939706,0.0,1.0
1,0.830072,1.315155,0.144345,6.972489,52.180608,35.661148,0.338904,0.072823,0.933834,0.862597,0.911406,1.338225,1.460531,1.025443,3.807510,1.752943,0.925465,0.0,1.0
2,0.899190,1.368519,0.233786,7.391479,58.765951,35.680414,0.299079,-0.150029,0.948801,0.865298,0.926187,1.375098,1.509800,1.027132,4.046311,1.790270,0.880337,0.0,1.0
3,0.920065,1.254861,0.665893,7.529317,60.679500,35.361833,0.241858,-0.253974,0.941032,0.874838,0.922516,1.368412,1.387347,1.026464,4.446514,1.787118,0.881928,0.0,1.0
4,0.857682,1.299201,0.300810,7.274365,55.875499,35.468512,0.275626,-0.116328,0.927939,0.857503,0.910001,1.332872,1.349506,1.025281,3.917411,1.777760,0.892314,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5755,0.431495,0.729295,0.303897,5.160629,27.887693,24.360120,0.077303,-0.412033,0.912760,0.829947,0.881141,1.326970,1.556884,1.023207,3.679571,1.739533,0.876301,47.0,0.0
5756,0.442420,0.796393,0.286033,5.231290,28.850624,24.196340,0.059848,-0.484124,0.914498,0.822261,0.886829,1.306199,1.541766,1.023451,4.140293,1.722263,0.841675,47.0,0.0
5757,0.454409,0.799703,0.306079,5.239264,28.749119,24.260379,0.053685,-0.481299,0.917962,0.832171,0.898225,1.316531,1.534797,1.023772,3.948767,1.738696,0.808856,47.0,0.0
5758,0.406959,0.655187,0.181755,5.094300,26.996555,24.222178,0.136604,-0.392639,0.917070,0.836731,0.893496,1.311565,1.510447,1.023607,3.594409,1.735797,0.820437,47.0,0.0


In [9]:
features_df[features_df["Patient Number"]==0][features_df["Label"]==1]

C:\Users\nimis\AppData\Local\Temp\ipykernel_19960\758833245.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  features_df[features_df["Patient Number"]==0][features_df["Label"]==1]


,mean_PSD,STD_PSD,A_mean,A_STD,A_Var,A_range,A_skew,A_kurtosis,Permutation_E,Spectral_E,SVD_E,Approximate_E,Sample_E,Petrosian_FD,Katz_FD,Higuchi_FD,Detrended fluctuation analysis,Patient Number,Label
0,0.666708,1.129708,0.522807,6.610550,46.226834,35.296581,0.269685,0.063777,0.921589,0.826680,0.879714,1.297318,1.426704,1.024025,3.909459,1.712382,0.939706,0.0,1.0
1,0.830072,1.315155,0.144345,6.972489,52.180608,35.661148,0.338904,0.072823,0.933834,0.862597,0.911406,1.338225,1.460531,1.025443,3.807510,1.752943,0.925465,0.0,1.0
2,0.899190,1.368519,0.233786,7.391479,58.765951,35.680414,0.299079,-0.150029,0.948801,0.865298,0.926187,1.375098,1.509800,1.027132,4.046311,1.790270,0.880337,0.0,1.0
3,0.920065,1.254861,0.665893,7.529317,60.679500,35.361833,0.241858,-0.253974,0.941032,0.874838,0.922516,1.368412,1.387347,1.026464,4.446514,1.787118,0.881928,0.0,1.0
4,0.857682,1.299201,0.300810,7.274365,55.875499,35.468512,0.275626,-0.116328,0.927939,0.857503,0.910001,1.332872,1.349506,1.025281,3.917411,1.777760,0.892314,0.0,1.0
5,0.794143,1.160293,0.073788,6.897319,50.121595,35.587001,0.267323,-0.006535,0.926644,0.862554,0.899839,1.337240,1.519218,1.024806,3.849666,1.770790,0.897325,0.0,1.0
6,0.656688,0.904526,0.123636,6.402079,43.412517,35.199528,0.162036,-0.038449,0.931764,0.856280,0.895633,1.334091,1.608560,1.025023,3.652884,1.759575,0.853197,0.0,1.0
7,0.676507,1.028360,0.166821,6.343888,42.401216,34.596301,0.190980,0.095129,0.928132,0.852339,0.900264,1.313403,1.494473,1.024740,3.901611,1.745673,0.890382,0.0,1.0
8,0.684420,0.956483,0.227338,6.363585,42.515279,35.220903,0.220413,0.145898,0.928682,0.862351,0.907298,1.320556,1.511250,1.024814,3.791353,1.775729,0.881367,0.0,1.0
9,0.582485,0.756750,0.145528,6.084821,38.962568,34.651556,0.157350,0.064192,0.930437,0.864077,0.899556,1.331328,1.621139,1.024943,3.520683,1.768588,0.842887,0.0,1.0


In [10]:
features_df.to_csv("features_extracted.csv",index=False)
print("All Features were extracted and saved in features_extracted.csv file")

All Features were extracted and saved in features_extracted.csv file
